In [1]:
{-# LANGUAGE GADTs #-}
{-# LANGUAGE OverloadedStrings #-}
import Grisette

# Solver-Aided Programming with Grisette

## Introduction

Grisette is a power Haskell library that enables solver-aided programming. It allows you to use off-the-shelf constraint solvers to reason about programs by introducing symbolic types and performing symbolic execution.

In this tutorial, you will learn about

- how to use symbolic types in Grisette, and
- how to call a solver to reason about the symbolic types, and
- how to construct a small verifier in Grisette.

## Symbolic Types

Let's start with a simple Haskell code snippet that processes two input `a` and `b`, and returns `a + a + b`.

In [2]:
concreteFunc :: Integer -> Integer -> Integer
concreteFunc a b = a + a + b

When applied to concrete values like 10 and 20, `concreteFunc` will return the concrete result.

In [3]:
concreteFunc 10 20

40

Grisette extends the programming model with symbolic types. In the following code, the `SymInteger` represents a symbolic integer, which can be a *symbolic term* over *symbolic constants*.
A *symbolic constant* is a placeholder for a constant value and can be created using its name with the `OverloadedStrings` extension.

Here is an example of the symbolic function that constructs a symbolic term.

In [4]:
symbolicFunc :: SymInteger -> SymInteger -> SymInteger
symbolicFunc a b = a + a + b

When applied to symbolic values, `symbolicFunc` will return a symbolic result. The result of `symbolicFunc "a" "b"` is the symbolic term `(+ (+ a a) b)`. Note that the symbolic constants with the same name are the same symbolic constant.

In [5]:
symbolicFunc "a" "b"

(+ (+ a a) b)

## Solver-Aided Reasoning

The power of symbolic values lies in their ability to be translated to off-the-shelf solvers for reasoning about programs.
Let's consider the following question:

- For what input `a` is the result of `symbolicFunc a 20` equal to 40?

To answer this, we construct a symbolic boolean term asserting that `symbolicFunc a 20` is equal to 40.

In [6]:
a :: SymInteger
a = "a"
print $ symbolicFunc a 20 .== 40
Right model <- solve (precise z3) $ symbolicFunc a 20 .== 40
print model

(= (+ a a) 20)

Model {a -> 10 :: Integer}

Note that we use `.==` instead of `==` to construct the symbolic term. The `solve` function sends the term to the solver (in this case, the Z3 solver) and returns either a model or a failure.
It treats all the symbolic constants in the boolean term as existentially quantified and tries to find assignments for them, and
a model is a mapping from symbolic constants to concrete constants that, when substituted into the boolean term, evaluates to true.

We can use the model to evaluate a symbolic term and perform the substitution.

In [7]:
print (evaluateSymToCon model a :: Integer)
print (evaluateSymToCon model (symbolicFunc a 20) :: Integer)
print (evaluateSymToCon model (symbolicFunc a 20 .== 40) :: Bool)

10

40

True

Sometimes, the solver may not find a model or may conclude that no such model exists. For example, if we ask the solver whether there exists an integer `a` such that `symbolicFunc a 20` is equal to 39, the solver will conclude that the constraint cannot be satisfied:

In [8]:
r <- solve (precise z3) $ symbolicFunc a 20 .== 39
print r

Left Unsat

The result, in this case, would be `Left Unsat`, indicating that the constraint is unsatisfiable.

## A Small Expression Equivalence Verifier

Now that we have a basic understanding of Grisette's symbolic types and solver-aided reasoning, let's dive into building a practical application: an expression equivalence verifier. This tool can be particularly useful when developing an optimizing compiler that performs term rewrites. By verifying the equivalence of expressions before and after the rewrite, you can ensure the correctness of your optimization rules.

We'll start by defining a small language for our expressions:

```
E -> I int
   | B bool
   | Add E E
   | Mul E E
   | Eq E E
```

Using GADTs (Generalized Algebraic Data Types), we can define well-typed instances of our expression language as follows:

(If you aren't familiar with GADTs, you may refer to https://en.wikibooks.org/wiki/Haskell/GADT. Our example is also adapted from the expression language defined there.)

In [9]:
data Expr a where
  I :: SymInteger -> Expr SymInteger
  B :: SymBool -> Expr SymBool
  Add :: Expr SymInteger -> Expr SymInteger -> Expr SymInteger
  Mul :: Expr SymInteger -> Expr SymInteger -> Expr SymInteger
  Eq :: SEq a => Expr a -> Expr a -> Expr SymBool

Note the `SEq` constraint in the `Eq` constructor. This is a type class provided by Grisette that enables the use of `.==` or `./=` operators for symbolic equality and inequality comparisons.

Next, let's define an evaluation function for our expression language:

In [10]:
eval :: Expr a -> a
eval (I n) = n
eval (B b) = b
eval (Add e1 e2) = eval e1 + eval e2
eval (Mul e1 e2) = eval e1 * eval e2
eval (Eq e1 e2) = eval e1 .== eval e2

With our language and evaluation function in place, we can now build the equivalence verifier. To prove that two expressions are equivalent, we need to show that the following property $p_1$ holds. Here, $\mathrm{consts}(e)$ is the set of all symbolic constants in the expression $e$.

$p_1=\forall~\mathrm{constant}\in \mathrm{consts}(e_1), \mathrm{consts}(e_2). \mathrm{eval}(e_1) = \mathrm{eval}(e_2)$

However, solvers typically struggle with efficiently handling universally quantified formulas. To circumvent this, we can transform $p_1$ into an existentially quantified formula $p_2$, such that $p_2$ is unsatisfiable implies that $p_1$ is a tautology:

$p_2=\exists~\mathrm{constant}\in \mathrm{consts}(e_1), \mathrm{consts}(e_2). \mathrm{eval}(e_1) \neq\mathrm{eval}(e_2)$

Here's the implementation of the equivalence verifier:

In [11]:
verifyEquivalent :: (EvaluateSym a, SEq a, Show a) => Expr a -> Expr a -> IO ()
verifyEquivalent e1 e2 = do
  res <- solve (precise z3) $ eval e1 ./= eval e2
  case res of
    Left Unsat -> putStrLn "The two expressions are equivalent"
    Left err -> putStrLn $ "The solver returned unexpected response: " <> show err
    Right model -> do
      putStrLn "The two expressions are not equivalent, under the model:"
      print model
      putStrLn $ "lhs evaluates to: " <> show (evaluateSym False model $ eval e1)
      putStrLn $ "rhs evaluates to: " <> show (evaluateSym False model $ eval e2)

The `EvaluateSym` type class used here is another Grisette type class that enables the substitution of symbolic constants with concrete values from a model. `evaluateSym False` substitutes only the symbolic constants present in the model, while `evaluateSym True` assigns default concrete values to any unmentioned constants.

Let's put our verifier to the test! First, we'll prove the distributive property of multiplication over addition:

In [12]:
verifyEquivalent
  (Mul (I "a") (Add (I "b") (I "c")))
  (Add (Mul (I "a") (I "b")) (Mul (I "a") (I "c")))

The two expressions are equivalent

Next, we'll attempt to prove an incorrect equivalence:

In [13]:
verifyEquivalent
  (Add (I "a") (Mul (I "b") (I "c")))
  (Mul (Add (I "a") (I "b")) (Add (I "a") (I "c")))

The two expressions are not equivalent, under the model:
Model {a -> -3 :: Integer, b -> 2 :: Integer, c -> 7 :: Integer}
lhs evaluates to: 11
rhs evaluates to: -4

As expected, the solver will provide a counterexample model demonstrating that the expressions are not equivalent.

Finally, let's verify a simple boolean equivalence (`con` converts a concrete boolean value to a symbolic boolean term):

In [14]:
verifyEquivalent (Eq (I "a") (I "a")) (B $ con True)

The two expressions are equivalent

## Conclusion

In this tutorial, we embarked on an exciting journey into the world of solver-aided programming with Grisette. We learned how Grisette extends the Haskell programming model with symbolic types, allowing us to represent unknown or uncertain values in our programs.

By introducing symbolic constants and leveraging the power of SMT solvers like Z3, we discovered how to reason about our programs and find concrete values that satisfy given constraints. We explored the `SymInteger` and `SymBool` typs. These symbolic types serve as the foundation for symbolic reasoning in Grisette.

Moreover, we delved into the concept of model generation, where the solver provides a mapping from symbolic constants to concrete values that make our symbolic expressions evaluate to a desired outcome.

To showcase the potential of Grisette, we built a small expression equivalence verifier. This example demonstrated how we can leverage symbolic execution and SMT solving to prove the equivalence of two expressions, or find counterexamples when they are not equivalent.

However, this is just the tip of the iceberg. Grisette offers a rich set of features and abstractions that enable us to tackle more complex problems and build sophisticated solver-aided tools. In future tutorials, we will explore these advanced concepts and learn how to harness the full power of Grisette.

As you continue your journey with Grisette, you will discover a whole new paradigm of programming, where the lines between traditional development and formal methods blur. By embracing symbolic reasoning and solver-aided techniques, you will unlock new possibilities for creating reliable, correct, and efficient software.